In [3]:
import os
import json
import pandas as pd
import traceback

In [4]:
# to Access OpenAI_API from langchain that is our llm
from langchain.chat_models import ChatOpenAI

In [5]:
# add your .env file then use the below code to load your API Key
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# temperature that take a value between 0 nad 2 ( near 0 is less creative)
llm=ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3
)

e:\Python-27122023\Learning\MCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON={
    "1":{
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

     "2":{
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
     "3":{
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}    
"""

In [12]:
# design your Prompt template
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [13]:
quiz_chain=LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
)

e:\Python-27122023\Learning\MCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the complexity
if the quiz is not at per with the cognitive and analytical of the student,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

check from an expert English Writer of the above quiz:
"""

In [15]:
# design your Prompt template2
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [16]:
review_chain=LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
)

In [17]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
)

In [18]:
file_path=r"E:\Python-27122023\Learning\MCQ\data.txt"

In [19]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [20]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [21]:
# serialize the dictionary to json format
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [23]:
#How to setup Token Usage Tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

e:\Python-27122023\Learning\MCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2204
Prompt Tokens:1775
Completion Tokens:429
Total Cost:0.0035205


In [25]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [26]:
quiz=response.get("quiz")

In [36]:
quiz

'\n{"1": {"mcq": "What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?", "options": {"a": "Artificial Intelligence", "b": "Machine Learning", "c": "Neural Networks", "d": "Deep Learning"}, "correct": "b"}, \n"2": {"mcq": "Who published the book \'The Organization of Behavior\' in 1949, introducing a theoretical neural structure?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "b"}, \n"3": {"mcq": "What was the name of the experimental \'learning machine\' developed by Raytheon Company in the early 1960s?", "options": {"a": "Cyberdyne", "b": "Cyberspace", "c": "Cybertron", "d": "Cybernet"}, "correct": "c"}, \n"4": {"mcq": "Who provided a formal definition of the algorithms studied in the machine learning field?", "options": {"a": "Donald Hebb", "b": "Walter Pitts", "c": "Tom M. Mitchell", "d": "Warren McCulloch"}, "correct": "c"}, \n"5": {"mcq": "What was the 

In [41]:
quiz=json.loads(quiz)

In [42]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ": mcq , "Choices":options, "Correct":correct})

In [43]:
quiz_table_data

[{'MCQ': 'What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a:Artificial Intelligence | b:Machine Learning | c:Neural Networks | d:Deep Learning',
  'Correct': 'b'},
 {'MCQ': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'Choices': 'a:Arthur Samuel | b:Donald Hebb | c:Walter Pitts | d:Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a:Cyberdyne | b:Cyberspace | c:Cybertron | d:Cybernet',
  'Correct': 'c'},
 {'MCQ': 'Who provided a formal definition of the algorithms studied in the machine learning field?',
  'Choices': 'a:Donald Hebb | b:Walter Pitts | c:Tom M. Mitchell | d:Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': 'What was the main success of neural networks research in the mid-1980s?',
  'Choices': 'a:Reinvention

In [45]:
quiz=pd.DataFrame(quiz_table_data)

In [46]:
quiz.to_csv("machine_Learning.csv",index=False)